In [1]:
import pandas as pd
import numpy as np
from data_preprocess import data_cleaning, data_spliting, scaling  ### Dont forget to restart the Kernel and re-import if any changes has been made to the methods ###

ds = pd.read_csv("D:/work_dir/Datasets/LoRa_anomaly-detection/RSSI_fingerprinting/antwerp_ds_weather-data.csv", index_col=0)

In [2]:
x_rssi, y = data_cleaning(ds, wathear_data=False)

x_weather, y = data_cleaning(ds, wathear_data=True)

Size before cleaning:  (123528, 109)
Size after cleaning:  (114011, 109)
Size before cleaning:  (123528, 109)
Size after cleaning:  (114011, 109)


In [3]:
# make 5 different train, test and validation sets 
# the result is going to be the average of these 

In [4]:
x_rssi_scaled = scaling(scaler='MinMax', x=x_rssi)
x_w_scaled = scaling(scaler='MinMax', x=x_weather)

x_rssi_train, x_rssi_test, x_rssi_val, y_rssi_train, y_rssi_test, y_rssi_val = data_spliting(x_rssi_scaled, y, 42)
x_w_train, x_w_test, x_w_val, y_w_train, y_w_test, y_w_val = data_spliting(x_w_scaled, y, 42)

In [6]:
from sklearn.neighbors import KNeighborsRegressor

model_rssi = KNeighborsRegressor(algorithm='ball_tree')

model_rssi.fit(x_rssi_train, y_rssi_train)

y_rssi_pred = model_rssi.predict(x_rssi_test)

In [7]:
from performance_eval import *

error_stats(y_rssi_test, y_rssi_pred)

400.0053201257178

In [9]:
model_weather = KNeighborsRegressor(algorithm='ball_tree')

model_weather.fit(x_w_train, y_w_train)
y_w_pred = model_weather.predict(x_w_test)
error_stats(y_w_test, y_w_pred)

368.1144223356488